<h1>Chess AI: Min Max Algorithm</h1><br>

First off we're using python-chess version 0.15.3 to handle<br>
the board, valid moves, etc. That way we can focus on the AI element.<br>

<b>python-chess:</b>
- https://python-chess.readthedocs.io/en/v0.15.0/core.html

In [ ]:
#import the python-chess library
import chess

Next we read from the config.txt file so that we can see how<br>
many moves we're allowed to make before the game ends due to<br>
turn limitation. <br>

In [ ]:
#read in the turn limit
with open('config.txt', 'r') as f:
    turnLim = int(f.readline())
print("The maximum number of moves for a player is: " + str(turnLim))

<h2>1.1 Setting up the Board</h2><br>

The initial state of the board being set up is done as follows:<br>
<font color="red">chess.Board('1n2k3/8/8/8/8/8/8/R3K1N1 w - - 0 0')</font><br>

The numbers represents empty squares starting from the top left of the board<br>
going line by line from top left to bottom right. Capital letters represent<br>
white pieces and lower case represent black pieces. The w signifies it's white's<br>
turn. 

In [ ]:
def setupBoard(*args):
    if len(args) == 2:
        moveString = args[1]
        board = args[0]
        if(moveString[0] == "X"):
            color = 1 #White
        elif moveString[0] == "Y":
            color = 0 #Black
        if moveString[2] == "K": 
            piece_type = 6 #King
        if moveString[2] == "R":
            piece_type = 4 #Rook
        if moveString[2] == "N":
            piece_type = 2  #Knight  
        p = board.pieces(piece_type, color)
        uci = chess.SQUARE_NAMES[list(p)[0]] + moveString[4:]
        move = chess.Move.from_uci(uci)
        if move in board.legal_moves:
            board.push(move)
            return True
        else:
            return False
    else:
        return chess.Board('1n2k3/8/8/8/8/8/8/R3K1N1 w - - 0 0')

Now we can print an ASCII representation of the board.

In [ ]:
b = setupBoard()
print(b)

<h2>1.2 Displaying the Board</h2><br>
Now we need to import two functions for displaying the board.<br>
- display
- SVG

In [ ]:
#Import display for displaying an svg representation of the board
from IPython.display import display, SVG    

In [ ]:
def showMove(board, player):
    display(SVG(board._repr_svg_()))
    if player == "X":
        file = "log_x.txt"
    if player == "Y":
        file = "log_y.txt"
    if player == "START":
        return
    with open(file, "a") as f:
        if board != chess.Board('1n2k3/8/8/8/8/8/8/R3K1N1 w - - 0 0'):
            move = board.peek()
            piece = str(board.piece_at(move.to_square)).upper()
            if str(board.piece_at(move.to_square)).islower():
                player = "Y" #Black Player is lower case
            else:
                player = "X" #White Player is upper case
            string = player +':'+ piece +':'+ move.uci()[2:]
            f.write(string + '\n')

Now when the initial board is created we can show the Board in a<br>
nice graphical vector image representation!

In [ ]:
b = setupBoard()
showMove(b, player="X")
if setupBoard(b, "X:R:a8"):
    showMove(b, player="START")

<h2>1.3 Making a Move</h2>

In [ ]:
def move(board, uci):
    if type(uci) is chess.Move:
        board.push(uci)
        return True
    else:
        move = chess.Move.from_uci(uci)
        if move in board.legal_moves:
            board.push(move)
            return True
    return False

In [ ]:
move(b, "a1a8")
showMove(b, "X")

<h2>1.4a Dealing with Move Checking</h2>

By importing check4Move() we can do a crossplatform<br>
check to see if an opponant has moved by doing a simple<br> 
polling method.

In [ ]:
from filehandling import *

<h2>1.4b Making sure log files are empty</h2>

To make sure that the log_x and log_y text files are<br>
empty. We go ahead and open them up at the begining of<br>
game and zero out.<br>

The function <font color="red">emptyLogFiles()</font> does this.

<h2>1.5 Checking for End Game</h2>

Must check to see if the game is finished. If it is<br>
we must return a proper end state string to tell for<br>
what reason it ended.

In [ ]:
def gameOver(board):
    if board.is_stalemate():
        return True
    if board.is_checkmate():
        return True
    return False

def endResult(board):
    if board.is_stalemate():
        return "stalemate"
    if board.is_checkmate():
        return "checkmate"

In [ ]:
from chessAI import *

<h2>1.6 Playing the Game</h2>

In [ ]:
def play(n, player):   
    board = setupBoard() #Initialize the board
    showMove(board, "X") #Display starting position
    for i in range(1, n):
        if gameOver(board):
            return endResult(board)
        if player == "X":
            print("White's Turn: " + str(i))
            #uci = input("Player X: ")
            uci = randomPlayer(board)
            move(board, uci)
            showMove(board, player)
            s = check4Move(player, i)
            if setupBoard(board, s):    
                showMove(board, player)
                if gameOver(board):
                    return endResult(board)
            else:
                return "Invalid Move made by Player Y"
        if player == "Y":
            s = check4Move(player, i)
            if setupBoard(board, s):
                print("White's Turn: " + str(i))
                showMove(board, player)
                if gameOver(board):
                    return endResult(board)
            else:
                return "Invalid Move made by Player X"
            print("Black's Turn: " + str(i))
            #uci = input("Player Y: ")
            uci = randomPlayer(board)
            move(board, uci)
            showMove(board, player)
    return "Maximum # of moves reached"
            
            

In [ ]:
result = play(turnLim, "X")

In [ ]:
print(result)